# Exercises

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pyspark
import pyspark.sql
import pyspark.ml
from pyspark.sql.functions import *

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from wrangle import wrangle_311

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

df = wrangle_311(spark)
print("\ndf shape: (%d, %d)\n" % (df.count(), len(df.columns)))
df.show(1, vertical=True)

[wrangle.py] reading case.csv
[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features
[wrangle.py] joining departments

df shape: (841704, 20)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                

1. Use the .randomSplit method to split the 311 data into training and test sets.

In [3]:
train, test = df.randomSplit([0.8, 0.2], seed=830)

2. Create a classification model to predict whether a case will be late or not (i.e. predict case_late). Experiment with different combinations of features and different classification algorithms.

In [4]:
train.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



In [5]:
rf = pyspark.ml.feature.RFormula(formula="case_late ~ department + council_district").fit(train)
train_input = rf.transform(train)
train_input.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 features             | (17,[2,8],

In [6]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()
lr_fit = lr.fit(train_input)
lr_fit

LogisticRegressionModel: uid = LogisticRegression_ec60926d2dda, numClasses = 2, numFeatures = 17

In [7]:
[x for x in dir(lr_fit.summary) if not x.startswith('_')]

['accuracy',
 'areaUnderROC',
 'fMeasureByLabel',
 'fMeasureByThreshold',
 'falsePositiveRateByLabel',
 'featuresCol',
 'labelCol',
 'labels',
 'objectiveHistory',
 'pr',
 'precisionByLabel',
 'precisionByThreshold',
 'predictionCol',
 'predictions',
 'probabilityCol',
 'recallByLabel',
 'recallByThreshold',
 'roc',
 'totalIterations',
 'truePositiveRateByLabel',
 'weightedFMeasure',
 'weightedFalsePositiveRate',
 'weightedPrecision',
 'weightedRecall',
 'weightedTruePositiveRate']

In [8]:
lr_fit.summary.areaUnderROC

0.6364818931212122

In [9]:
lr_fit.summary.accuracy

0.8889640632117142

In [10]:
lr_fit.summary.recallByLabel

[0.9988700111661618, 0.021015669346428525]

In [11]:
lr_fit.summary.precisionByLabel

[0.8895955314780988, 0.7019400352733686]

In [12]:
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(lr_fit.transform(rf.transform(test)))
test_auc

0.6352165112610344

In [13]:
test_input = rf.transform(test)

test_input.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127341           
 case_opened_date     | 2018-01-01 06:59:00  
 case_closed_date     | 2018-01-02 15:32:00  
 case_due_date        | 2018-01-17 08:30:00  
 case_late            | false                
 num_days_late        | -14.70649306         
 case_closed          | true                 
 service_request_type | Front Or Side Yar... 
 SLA_days             | 16.0628588           
 case_status          | Closed               
 source_id            | svcCRMSS             
 request_address      | 10133  FIGARO CAN... 
 council_district     | 004                  
 num_weeks_late       | -2.10092758          
 zipcode              | 78251                
 case_age             | 219                  
 days_to_closed       | 1                    
 case_lifetime        | 1                    
 department           | DSD/Code Enforcement 
 dept_subject_to_SLA  | true                 
 features             | (17,[0,11]

In [14]:
# confusion matrix for the test data
(lr_fit.transform(test_input)
 .select('case_late', 'department', 'council_district', 'label', 'probability', 'prediction')
 .groupby('prediction') # predicted == rows
 .pivot('label') # actual values are columns
 .count()
 .show())

+----------+------+-----+
|prediction|   0.0|  1.0|
+----------+------+-----+
|       0.0|148802|18332|
|       1.0|   163|  418|
+----------+------+-----+



3. Create a regression model to predict how many days late a case will be (i.e. predict num_days_late). Experiment with different combinations of features and different regression algorithms.

In [18]:
train.show(1,vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



In [48]:
# nb: spark's rformula does encoding
rf = pyspark.ml.feature.RFormula(formula="num_days_late ~ SLA_days").fit(train)

rf

RFormula_ac83af72cd51

In [49]:
rf.transform(train).show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 features             | [999.0]   

In [50]:
train_input = rf.transform(train).select('features', 'label')
train_input.show(5)

+-------------+-------------------+
|     features|              label|
+-------------+-------------------+
|      [999.0]| -998.5087616000001|
|[4.322222222]|-2.0126041669999997|
|[4.320729167]|       -3.022337963|
|[16.29188657]|       -15.01148148|
|      [0.125]|0.37216435200000003|
+-------------+-------------------+
only showing top 5 rows



In [51]:
lr = pyspark.ml.regression.LinearRegression()
lr

LinearRegression_efbffde9495b

In [52]:
lr_fit = lr.fit(train_input)

Py4JJavaError: An error occurred while calling o1263.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 268.0 failed 1 times, most recent failure: Lost task 3.0 in stage 268.0 (TID 2015, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<SLA_days:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Itera

In [37]:
lr_fit.transform(train_input).show(5)

Py4JJavaError: An error occurred while calling o914.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 247.0 failed 1 times, most recent failure: Lost task 0.0 in stage 247.0 (TID 1981, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:255)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.a

In [38]:
training_summary = lr_fit.summary

In [39]:
training_summary.r2, training_summary.rootMeanSquaredError

AttributeError: 'BinaryLogisticRegressionTrainingSummary' object has no attribute 'r2'

In [40]:
training_summary.meanAbsoluteError, training_summary.meanSquaredError

AttributeError: 'BinaryLogisticRegressionTrainingSummary' object has no attribute 'meanAbsoluteError'

In [41]:
training_summary.residuals.show(5)

AttributeError: 'BinaryLogisticRegressionTrainingSummary' object has no attribute 'residuals'

In [42]:
test_input = rf.transform(test)
lr_fit.transform(test_input).show(4)

Py4JJavaError: An error occurred while calling o1010.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 249.0 failed 1 times, most recent failure: Lost task 0.0 in stage 249.0 (TID 1983, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:255)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.

In [ ]:
evaluator = pyspark.ml.evaluation.RegressionEvaluator()
rmse = evaluator.evaluate(lr_fit.transform(test_input))
rmse